In [1]:
%load_ext tensorboard
import tensorflow as tf
import datetime
import keras

import numpy as np
import pandas as pd
from keras.models import Sequential, Model, load_model
from keras.layers import Input ,Dense, Dropout, Activation, LSTM, Conv1D
from keras.layers import Lambda, Convolution1D, MaxPooling1D, Flatten, Reshape, BatchNormalization
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.metrics import categorical_crossentropy, binary_crossentropy

#from spp.SpatialPyramidPooling import SpatialPyramidPooling

#For data saving
import pickle
import random
#other imports
import gzip
import glob
import os
import keras.backend as K
import os
import time 
#cwd = os.path.dirname(os.path.realpath("SURF_001_TwoClass13.ipynb"))

In [ ]:
dense_layer = 1
layer_size = 64 
conv_layer = 2
seq_length = 1000
base_pair = 4
num_strides = 1
pool_size = 4

epochs = 100
batch_size = 1
val_batch_size = 40

In [ ]:
# Loads the weights


checkpoint_path = "../training_1/dataset14_run1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                             save_weights_only=True,
                                             verbose=1)

earlystop = tf.keras.callbacks.EarlyStopping(
monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto',
baseline=None, restore_best_weights=False)

#Model
model = tf.keras.models.Sequential([      

    tf.keras.layers.Conv1D(64, (4), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(64, (3), activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')       
])
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

#model.load_weights("../training_1/dataset14_run1.ckpt")

# Re-evaluate the model
#loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
#print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
with open('../partition_train_val_a.dat', 'rb') as f:
    partition = pickle.load(f)
train_samples_IDs = partition['train']
val_samples_IDs = partition['validation']
IDs = partition['labels']

In [ ]:
predicted = []
Y_val_expected = []
count_input_val = 0
num_val_seq = len(val_samples)
done = False
while count_input_val < num_val_seq and not(done):
    X_val = np.zeros((1, 1000, 4), dtype=np.int8)
    if count_input_val < (num_val_seq - 1):
        for i in range(0, 1):
            a = val_samples[count_input_val]
            X_val[i, :, :] = np.load('../model_11_data_4/' + val_samples[count_input_val] + '.npy') 
            count_input_val += 1
        predict = (model.predict_classes(X_val))
        for label in predict:
            predicted.append(label)
    else:
        done = True
    if count_input_val == 16200:
        done = True
    print(count_input_val)
print(len(predicted))
print(predicted[0])

In [ ]:
expected = []

for label in val_samples:
    expected.append(labels[label][0][0])
print(expected[0])
print(len(expected))


In [ ]:
# Example of a confusion matrix in Python
from sklearn.metrics import confusion_matrix
 
results = confusion_matrix(expected[0:13400], predicted)
print(results)
# Printing the precision and recall, among other metrics
#print(metrics.classification_report(y_act, y_pred, labels=["a", 
#"b"]))

In [ ]:
import seaborn as sns

sns.heatmap(results/np.sum(results), annot=True, 
            fmt='.2%', cmap='Blues')
print(model.summary)